# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pprint import pprint
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv from Part I of the homework
weather_df = pd.read_csv("../Resources/weather_data_part-I.csv")
len(weather_df)
weather_dropped_df = weather_df.dropna()
len(weather_dropped_df)
print(f"Check df lenfth before and after dropping na values -\
 {len(weather_df)} vs. {len(weather_dropped_df)}") # Check if any na values
weather_df = weather_dropped_df
weather_df.head()

Check df lenfth before and after dropping na values - 541 vs. 541


,City,Temperature (DegC),Cloudiness (%),Latitude,Longitude,Humidity (%),Wind Speed (mps)
0,Kulhudhuffushi,29.10,50,6.62,73.07,78,7.29
1,Kaitangata,7.22,0,-46.28,169.85,58,1.79
2,Kahului,24.26,20,20.89,-156.47,51,12.30
3,Bubaque,26.99,94,11.28,-15.83,83,5.03
4,Albany,7.41,7,42.60,-73.97,68,1.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmap
gmaps.configure(api_key=g_key)

In [4]:
# Use Lat and Lng as locations and Humidity as the weight
locations = weather_df[['Latitude', 'Longitude']]
humidity  = weather_df['Humidity (%)']

In [5]:
# Add Heatmap layer to map
# fig = gmaps.figure()
fig = gmaps.figure(center=(40.0, -5.0), zoom_level=2)

In [6]:
# Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=2)

In [7]:
# Add layer
fig.add_layer(heat_layer)

In [8]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
warm_df = weather_df[weather_df['Temperature (DegC)'] > 25]
warm_and_noCloud_df = warm_df.loc[(warm_df['Cloudiness (%)'] <= 80)]
warm_and_noCloud_df.head()

,City,Temperature (DegC),Cloudiness (%),Latitude,Longitude,Humidity (%),Wind Speed (mps)
0,Kulhudhuffushi,29.10,50,6.62,73.07,78,7.29
21,Kodinsk,26.91,71,58.69,99.18,46,5.09
24,San Patricio,26.04,1,19.22,-104.70,83,2.20
30,Bengkulu,29.18,2,-3.80,102.27,61,1.84
34,Atuona,26.82,36,-9.80,-139.03,81,9.31


In [10]:
eur_df = warm_and_noCloud_df.loc[warm_and_noCloud_df['Longitude'] < 60]
eur_df.head()

,City,Temperature (DegC),Cloudiness (%),Latitude,Longitude,Humidity (%),Wind Speed (mps)
24,San Patricio,26.04,1,19.22,-104.70,83,2.20
34,Atuona,26.82,36,-9.80,-139.03,81,9.31
36,Faanui,27.28,36,-16.48,-151.75,76,4.14
41,Tautira,27.88,20,-17.73,-149.15,69,2.10
65,Alice Town,27.26,36,25.72,-79.30,75,7.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
types_df = pd.read_csv("Resources/ethnic_restr.csv")
types_df.head()

,ethnicity
0,chinese
1,cuban
2,czech
3,french
4,german


In [18]:
# set up additional columns to hold information
types_df['name'] = ""
types_df['address'] = ""
types_df['price_level'] = ""
types_df['rating'] = ""

types_df.head()

,ethnicity,name,address,price_level,rating
0,chinese,,,,
1,cuban,,,,
2,czech,,,,
3,french,,,,
4,german,,,,


In [57]:
# Setup query parameters
target_type = "restaurant"
locations = eur_df[['Latitude', 'Longitude']]
radius = 5000
#restr_type = 'chinese'
restr_type = 'restaurant'

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
keyword = 'chinese'

# Re-write into parameters dictionary
params = {
    'radius': 5000,
    'type': target_type,
    'key': g_key
}
restr_list        = []
restr_vcnt_list   = []
restr_rating_list = []
locs_list         = []
counties          = []

for index, row in locations.iterrows():
    response = requests.get(base_url, params).json()
    query_url = (f"{base_url}?location={row[0]},{row[1]}&radius={radius}&type={target_type}&key={g_key}")
    response = requests.get(query_url)
    response_data = response.json()
    results = response_data['results']
    try:
        print(f"Closest {target_type} restaurant is {results[0]['name']}.")
        restr_list.append(results[0]['name'])
        print(restr_list)
        restr_vcnt_list.append(results[0]['vicinity'])
        countries.append(results[0][])
        locs_list.append((row[0],row[1]))
        print(locs_list)
        if (results[0]['user_ratings_total'] == ""):
            print("Missing user ratings: ")
        else:
            restr_vcnt_df = pd.DataFrame({'Restaurant Name': restr_list,
                         'Vicinity': restr_vcnt_list})
            print(len(restr_vcnt_df))
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("---------------------------------------")
print("------------------END------------------")
    

Closest restaurant restaurant is Señor Froy´s.
['Señor Froy´s']
[(19.22, -104.7)]
1
---------------------------------------
Closest restaurant restaurant is Roulotte Heremoana.
['Señor Froy´s', 'Roulotte Heremoana']
[(19.22, -104.7), (-9.8, -139.03)]
2
---------------------------------------
Closest restaurant restaurant is Iaorana Gelato Homemade Italian Ice Cream, internet point (WiFi free).
['Señor Froy´s', 'Roulotte Heremoana', 'Iaorana Gelato Homemade Italian Ice Cream, internet point (WiFi free)']
[(19.22, -104.7), (-9.8, -139.03), (-16.48, -151.75)]
3
---------------------------------------
Closest restaurant restaurant is Le Bout du Monde.
['Señor Froy´s', 'Roulotte Heremoana', 'Iaorana Gelato Homemade Italian Ice Cream, internet point (WiFi free)', 'Le Bout du Monde']
[(19.22, -104.7), (-9.8, -139.03), (-16.48, -151.75), (-17.73, -149.15)]
4
---------------------------------------
Closest restaurant restaurant is Bimini Big John's.
['Señor Froy´s', 'Roulotte Heremoana', 'Iaora

Closest restaurant restaurant is Darchin Restaurant.
['Señor Froy´s', 'Roulotte Heremoana', 'Iaorana Gelato Homemade Italian Ice Cream, internet point (WiFi free)', 'Le Bout du Monde', "Bimini Big John's", 'Taquería Lupita', 'Hotel Tesoro Los Cabos', 'Hukilau Lanai', "Hungry's Food Van", 'بيت البيتزا', 'Subway', 'Paradise Cove Luau', 'كافتيريا ناصير', 'لقمینو | Loqmino', 'birrieria El Shaddai', 'Little Caesars Pizza', 'Painted Pony Restaurant', 'Darchin Restaurant']
[(19.22, -104.7), (-9.8, -139.03), (-16.48, -151.75), (-17.73, -149.15), (25.72, -79.3), (15.74, -96.47), (22.89, -109.91), (22.08, -159.32), (18.22, -63.06), (26.33, 43.98), (32.66, -114.41), (21.35, -158.09), (25.02, 37.27), (28.75, 54.54), (25.38, -103.42), (39.05, -95.68), (37.1, -113.58), (35.57, 53.4)]
18
---------------------------------------
Closest restaurant restaurant is Makutano Restaurant & Bar.
['Señor Froy´s', 'Roulotte Heremoana', 'Iaorana Gelato Homemade Italian Ice Cream, internet point (WiFi free)', 'Le 

Closest restaurant restaurant is Manouchehri Traditional House And Hotel.
['Señor Froy´s', 'Roulotte Heremoana', 'Iaorana Gelato Homemade Italian Ice Cream, internet point (WiFi free)', 'Le Bout du Monde', "Bimini Big John's", 'Taquería Lupita', 'Hotel Tesoro Los Cabos', 'Hukilau Lanai', "Hungry's Food Van", 'بيت البيتزا', 'Subway', 'Paradise Cove Luau', 'كافتيريا ناصير', 'لقمینو | Loqmino', 'birrieria El Shaddai', 'Little Caesars Pizza', 'Painted Pony Restaurant', 'Darchin Restaurant', 'Makutano Restaurant & Bar', 'Restaurant Udiamoral', 'Subway', 'Rozna Muscat روزنة مسقط', 'مطعم مرزوقة', "Izzy's", 'حديقة ظلال إفريقيا', "McDonald's", 'Manouchehri Traditional House And Hotel']
[(19.22, -104.7), (-9.8, -139.03), (-16.48, -151.75), (-17.73, -149.15), (25.72, -79.3), (15.74, -96.47), (22.89, -109.91), (22.08, -159.32), (18.22, -63.06), (26.33, 43.98), (32.66, -114.41), (21.35, -158.09), (25.02, 37.27), (28.75, 54.54), (25.38, -103.42), (39.05, -95.68), (37.1, -113.58), (35.57, 53.4), (-7.

In [58]:
# set up additional columns to hold information
restr_vcnt_df = pd.DataFrame({
                            'Restaurant Name': restr_list,
                            'Vicinity': restr_vcnt_list,
                            'Coordinates': locs_list})
len(restr_vcnt_df)
#restr_vcnt_df.index += 1
restr_vcnt_df.to_csv("OutputData/Restaurants_in_proximity.csv", index=False)
restr_vcnt_df.head() 
len(restr_vcnt_df) # 33

33

In [66]:
# Get hotel coordinates into a list of tuples
# Load csv back into df
restr_df = pd.read_csv("OutputData/Restaurants_in_proximity.csv", index_col=None, usecols=None)
restr_df.index += 1
locs_list
markers = gmaps.marker_layer(locs_list)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
restos = restr_df['Restaurant Name']
coord_list = []
for resto in restos:
    print(resto)

1                                          Señor Froy´s
2                                    Roulotte Heremoana
3     Iaorana Gelato Homemade Italian Ice Cream, int...
4                                      Le Bout du Monde
5                                     Bimini Big John's
6                                       Taquería Lupita
7                                Hotel Tesoro Los Cabos
8                                         Hukilau Lanai
9                                     Hungry's Food Van
10                                          بيت البيتزا
11                                               Subway
12                                   Paradise Cove Luau
13                                       كافتيريا ناصير
14                                     لقمینو | Loqmino
15                                 birrieria El Shaddai
16                                 Little Caesars Pizza
17                              Painted Pony Restaurant
18                                   Darchin Res

In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{'Se√±or Froy¬¥s'}</dd>
<dt>City</dt><dd>{'City'}</dd>
<dt>Country</dt><dd>{'Country'}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in restr_vcnt_df.iterrows()]
locations = restr_vcnt_df[["Lat", "Lng"]]

KeyError: "'Se√±or Froy¬¥s'"

In [69]:
restr_vcnt_df

,Restaurant Name,Vicinity,Coordinates
0,Señor Froy´s,"Calle Filipinas 66, Barra de Navidad","(19.22, -104.7)"
1,Roulotte Heremoana,Taha Uku,"(-9.8, -139.03)"
2,"Iaorana Gelato Homemade Italian Ice Cream, int...","Vaitape, Bora Bora","(-16.48, -151.75)"
3,Le Bout du Monde,,"(-17.73, -149.15)"
4,Bimini Big John's,"Alice Town, North Bimini, North Bimini, Bahama...","(25.72, -79.3)"
5,Taquería Lupita,Av. Lázaro Cárdenas S/N junto a Materiales el ...,"(15.74, -96.47)"
6,Hotel Tesoro Los Cabos,"Boulevard Paseo de la Marina LT 9 y 10, Marina...","(22.89, -109.91)"
7,Hukilau Lanai,"520 Aleka Loop, Kapaʻa","(22.08, -159.32)"
8,Hungry's Food Van,The Valley,"(18.22, -63.06)"
9,بيت البيتزا,"3409 Umar Ibn Abdul Aziz Road, Buraydah","(26.33, 43.98)"


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{'Roulotte Heremoana'}</dd>
<dt>City</dt><dd>{'Taha Uku'}</dd>
<dt>Mexico</dt><dd>{'Mexico'}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in restr_vcnt_df.iterrows()]

locations = restr_vcnt_df[["Lat", "Lng"]]

KeyError: "'Roulotte Heremoana'"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
